## Installation Dependencies

In [1]:
%%capture
!pip install --upgrade -qqq uv
!pip install peft boto3 trl datasets
try: import numpy; install_numpy = f"numpy=={numpy.__version__}"
except: install_numpy = "numpy"
!uv pip install -qqq \
    "torch>=2.8.0" "triton>=3.4.0" {install_numpy} \
    "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
    "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
    torchvision bitsandbytes \
    git+https://github.com/huggingface/transformers \
    git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels

## Load Model

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 4096
dtype = None
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",
    dtype = dtype,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Gpt_Oss patching. Transformers: 4.56.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.
Unsloth: Gpt_Oss does not support SDPA - switching to fast eager.


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We now add LoRA adapters for parameter efficient finetuning - this allows us to only efficiently train 1% of all parameters.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


## Load Dataset

In [35]:
import boto3
import json
import pprint
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

s3 = boto3.client('s3', aws_access_key_id="",
                  aws_secret_access_key="")
END = "<|end|>"

def concatenate_json_data(bucket_name: str, prefix: str = '') -> list:
    all_data = []

    try:
        paginator = s3.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

        for page in pages:
            if "Contents" in page:
                for obj in page['Contents']:
                    if obj['Key'].endswith('.json'):
                        try:
                            response = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
                            content = response['Body'].read().decode('utf-8')
                            data = json.loads(content)
                            if isinstance(data.get('data'), list):
                              for item in data['data']:
                                if item["validation_result"]["is_valid"]:
                                  all_data.append({
                                      "text": f"Input: {item['processed']}\nOutput: {item['original']} {END}"
                                  })
                            else:
                                all_data.extend(data['data'])
                        except Exception as e:
                            print(f"Skipping file {obj['Key']} due to error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    return all_data

dataset = concatenate_json_data("fb-cleaned-data")

In [36]:
from datasets import Dataset
dataset = Dataset.from_list(dataset)

# Load Trainer

In [37]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4631 [00:00<?, ? examples/s]

In [38]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,631 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Step,Training Loss
1,3.516400
2,3.300000
3,2.841600
4,2.800100
5,2.511100
6,2.813300
7,2.355600
8,2.339900
9,2.846800


Step,Training Loss
1,3.516400
2,3.300000
3,2.841600
4,2.800100
5,2.511100
6,2.813300
7,2.355600
8,2.339900
9,2.846800
10,2.168300


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,631 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 3 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (3 x 4 x 1) = 12
 "-____-"     Trainable parameters = 3,981,312 of 20,918,738,496 (0.02% trained)


Step,Training Loss
1,2.475000
2,2.183000
3,2.597600
4,1.934800
5,2.056300
6,2.391500
7,2.154000
8,2.263100
9,2.091300
10,2.025500


## Inference

In [ ]:
FastLanguageModel.for_inference(model)

text = "That's so funny"
prompt = f"Input: {text}.\nOutput:"

enc = tokenizer(prompt, return_tensors="pt").to(model.device)

gen = model.generate(
    **enc,
    max_new_tokens=48,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

inp_len = enc["input_ids"].shape[-1]
completion = tokenizer.decode(gen[0, inp_len:], skip_special_tokens=True)
completion = completion.split(END)[0].strip().split("\n")[0]
print(completion)